In [38]:
import geopandas as gpd
import pandas as pd
import shapely.geometry as sg
import plotly.express as px
import plotly.graph_objects as go
import numpy as np



In [39]:
df_geral = {}

# Lendo os arquivos de 2019 a 2022 

for i in range(2019, 2023): 
    df_geral[i] = pd.read_csv(f'data\_{i}\_{i}.txt', sep='\t')

    # Retirando as linhas que possuem coordenadas nulas
    df_geral[i] = df_geral[i].drop(df_geral[i][df_geral[i]['COORD.GEO.X'] == 0].index)
    df_geral[i] = df_geral[i].drop(df_geral[i][df_geral[i]['COORD.GEO.Y'] == 0].index)

   # Filtrando pela província de La Victoria 
    df_geral[i] = df_geral[i][df_geral[i]['DISTRITO'] == 'LA VICTORIA']


df_geral[2019].head()

C:\Users\hyago\AppData\Local\Temp\ipykernel_17840\596211477.py:6: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\hyago\AppData\Local\Temp\ipykernel_17840\596211477.py:6: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\hyago\AppData\Local\Temp\ipykernel_17840\596211477.py:6: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



NRO_CLIENT     DISTRITO TARIFA  CATEGORIA  CNS_ACT_FP  CNS_ACT_HP SET  \
0          511  LA VICTORIA   BT5B        0.0       287.2         0.0   G   
9         1230  LA VICTORIA   BT5B        0.0       189.9         0.0   C   
13        1263  LA VICTORIA   BT5B        0.0       958.3         0.0   G   
15        1280  LA VICTORIA   BT5B        0.0       341.0         0.0   G   
19        1636  LA VICTORIA   BT5B        0.0       156.0         0.0  SL   

   ALIMENTAD      SED  SEC_TIPICO  COORD.GEO.X  COORD.GEO.Y  MES  
0       13.0  S-00433           1    279517.90   8666061.70    1  
9       19.0  S-00331           1    280230.44   8664222.03    1  
13      18.0  S-00036           1    279400.19   8665300.19    1  
15      13.0  S-00433           1    279484.79   8666131.13    1  
19       3.0  S-00303           1    281826.22   8665574.42    1

In [40]:
# Fazendo Pontos a partir das coordenadas, isso é necessário para o geopandas
# criar a coluna geometry do Geodataframe

points = {}

for i in range(2019, 2023):
    points[i] = df_geral[i].apply(lambda row: sg.Point(row['COORD.GEO.X'], row['COORD.GEO.Y']), axis=1)


c:\Users\hyago\miniconda3\Lib\site-packages\pandas\core\dtypes\cast.py:130: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.



In [41]:
geodf = {}

for i in range(2019, 2023):
    # Criando os Geodataframes a partir do dataframe e dos pontos criados
    geodf[i] = gpd.GeoDataFrame(df_geral[i], geometry=points[i])
    # Setando o sistema de coordenadas para o UTM 18S, que é o sistema de coordenadas dos dados
    geodf[i].crs = '32718'
    # Convertendo para o sistema de coordenadas geográficas WGS84
    # Isso é necessário para o plotly conseguir plotar os pontos
    geodf[i] = geodf[i].to_crs('EPSG:4326')



In [42]:
# Subtraindo 0.0022 do x e 0.0032 do y para que os pontos fiquem mais centralizados
# Obs: Isso é necessário pois o plotly não está plotando os pontos no local correto
# Obs2: Não sei porque isso está acontecendo, mas acredito que seja alguma conversão de sistema de coordenadas
# Obs3: Esses valores foram encontrados na tentativa e erro, não sei se são os valores corretos
for i in range(2019, 2023):
    geodf[i]['geometry'] = geodf[i]['geometry'].apply(lambda x: sg.Point(x.x - 0.0022, x.y - 0.0032))

In [43]:
# O quadrado escolhido para análise foram os 296 e 316 do outro código 
square_216 = [
    sg.Point(-77.0019956353078, -12.06870647082062), 
    sg.Point(-77.0019956353078, -12.065700086444597), 
    sg.Point(-77.00500201968381, -12.065700086444597), 
    sg.Point(-77.00500201968381, -12.06870647082062), 
    sg.Point(-77.0019956353078, -12.06870647082062)
]

square_316 = [
    sg.Point(-76.99898925093177, -12.06870647082062), 
    sg.Point(-76.99898925093177, -12.065700086444597), 
    sg.Point(-77.0019956353078, -12.065700086444597),
    sg.Point(-77.0019956353078, -12.06870647082062), 
    sg.Point(-76.99898925093177, -12.06870647082062)
    ]


In [44]:
# Irei filtrar os geodataframes de forma a restar apenas as linhas que possuem 
# pontos dentro do quadrado 216 e 316, além de criar a coluna "quadrado" que
# indica em qual quadrado o ponto está

for i in range(2019, 2023):
    geodf[i]['quadrado'] = geodf[i]['geometry'].apply(lambda x: '216' if sg.Point(x.x, x.y).within(sg.Polygon(square_216)) else '316' if sg.Point(x.x, x.y).within(sg.Polygon(square_316)) else 'outro')
    geodf[i] = geodf[i][geodf[i]['quadrado'] != 'outro']
    




In [45]:
# Juntarei os geodataframes de 2019 a 2022 em um único geodataframe, adicionando a coluna "datetime"
# que indica o ano e mês, representado pela coluna "MES", da linha no formato AAAA-MM

geodf_geral = gpd.GeoDataFrame()

for i in range(2019, 2023):
    geodf[i]["ANO"] = i

dfs_to_concat = [geodf[i] for i in range(2019, 2023)]
geodf_geral = gpd.GeoDataFrame(pd.concat(dfs_to_concat, ignore_index=True))

geodf_geral['datetime'] = geodf_geral.apply(lambda row: f'{row["ANO"]}-{row["MES"]}', axis=1)




In [46]:

# Excluindo as colunas que não serão utilizadas
geodf_geral = geodf_geral.drop(columns=['MES', 'ANO', 'COORD.GEO.Y', 'COORD.GEO.X', 'geometry', 'DISTRITO'])
geodf_geral.head()



NRO_CLIENT TARIFA  CATEGORIA  CNS_ACT_FP  CNS_ACT_HP SET ALIMENTAD  \
0       19762   BT5B        0.0       246.3         0.0  SL       3.0   
1       18711   BT5B        0.0       354.0         0.0  SL       3.0   
2       24617   BT5B        7.0      5004.5         0.0  SL       3.0   
3     1423425   BT5B        0.0       136.6         0.0  SL       3.0   
4       48851   BT5B        0.0       198.2         0.0  SL       4.0   

       SED  SEC_TIPICO quadrado datetime  
0  S-00304           1      216   2019-1  
1  S-00305           1      316   2019-1  
2  S-00305           1      316   2019-1  
3  C-08901           1      216   2019-1  
4  A-03646           1      216   2019-1

In [47]:
# Filtrarei o geodataframe de forma a restar apenas os clientes que aparecem em todos
# os meses, ou seja, que possuem 48 linhas, que é o número de meses de 2019 a 2022

geodf_geral = geodf_geral.groupby('NRO_CLIENT').filter(lambda x: len(x) == 48)

In [48]:
geodf_geral.to_csv('geodf_geral.csv', index=False)

In [49]:
geodf_geral = pd.read_csv('geodf_geral.csv')
geodf_geral.head()

NRO_CLIENT TARIFA  CATEGORIA  CNS_ACT_FP  CNS_ACT_HP SET  ALIMENTAD  \
0       19762   BT5B        0.0       246.3         0.0  SL        3.0   
1       18711   BT5B        0.0       354.0         0.0  SL        3.0   
2       24617   BT5B        7.0      5004.5         0.0  SL        3.0   
3     1423425   BT5B        0.0       136.6         0.0  SL        3.0   
4       48851   BT5B        0.0       198.2         0.0  SL        4.0   

       SED  SEC_TIPICO  quadrado datetime  
0  S-00304           1       216   2019-1  
1  S-00305           1       316   2019-1  
2  S-00305           1       316   2019-1  
3  C-08901           1       216   2019-1  
4  A-03646           1       216   2019-1

In [50]:
# Filtrando pela tarifa BT5B e pela categoria 0.0, consumo até 500 kWh

geodf_geral = geodf_geral[geodf_geral['TARIFA'] == ' BT5B']
geodf_geral = geodf_geral[geodf_geral['CATEGORIA'] == 0.0]

In [51]:
geodf_geral = geodf_geral.groupby('NRO_CLIENT').filter(lambda x: len(x) == 48)
geodf_geral["NRO_CLIENT"].value_counts()

NRO_CLIENT
19762      48
1570328    48
1625765    48
1709974    48
1683531    48
           ..
816562     48
816669     48
816674     48
868906     48
519464     48
Name: count, Length: 1022, dtype: int64

In [52]:
# Vou calcular a média de todos os clientes durante todo o período e colocar em um dataframe
# Para isso, vou agrupar por NRO_CLIENTE e depois calcular a média de consumo (CNS_ACT_FP)
df_medias = geodf_geral[['NRO_CLIENT', 'CNS_ACT_FP']].groupby('NRO_CLIENT').mean().reset_index()
df_medias.tail()

NRO_CLIENT  CNS_ACT_FP
1017     1916165  138.495833
1018     1920158  363.039583
1019     1923450  339.204167
1020     1923451  304.458333
1021     1926005  355.702083

In [53]:
# Utilizarei o dataframe de médias para filtrar o geodataframe geral, de forma a restar apenas
# os clientes que possuem uma média de consumo menor que 500 kWh

geodf_geral = geodf_geral.merge(df_medias, left_on='NRO_CLIENT', right_on='NRO_CLIENT')
geodf_geral = geodf_geral[geodf_geral['CNS_ACT_FP_y'] <= 250]
geodf_geral = geodf_geral[geodf_geral['CNS_ACT_FP_y'] >= 50]


In [54]:
geodf_geral.head()

NRO_CLIENT TARIFA  CATEGORIA  CNS_ACT_FP_x  CNS_ACT_HP SET  ALIMENTAD  \
96      1423425   BT5B        0.0         136.6         0.0  SL        3.0   
97      1423425   BT5B        0.0         136.4         0.0  SL        3.0   
98      1423425   BT5B        0.0         131.0         0.0  SL        3.0   
99      1423425   BT5B        0.0         147.0         0.0  SL        3.0   
100     1423425   BT5B        0.0         126.0         0.0  SL        3.0   

         SED  SEC_TIPICO  quadrado datetime  CNS_ACT_FP_y  
96   C-08901           1       216   2019-1       129.625  
97   C-08901           1       216   2019-2       129.625  
98   C-08901           1       216   2019-3       129.625  
99   C-08901           1       216   2019-4       129.625  
100  C-08901           1       216   2019-5       129.625

In [55]:
geodf_geral_216 = geodf_geral[geodf_geral['quadrado'] == 216]
geodf_geral_316 = geodf_geral[geodf_geral['quadrado'] == 316]

geodf_geral_216 = geodf_geral_216.groupby('NRO_CLIENT').filter(lambda x: len(x) == 48)
geodf_geral_316 = geodf_geral_316.groupby('NRO_CLIENT').filter(lambda x: len(x) == 48)

geodf_geral_216.reset_index(inplace=True, drop=True)
geodf_geral_316.reset_index(inplace=True, drop=True)


In [56]:
geodf_geral_216["NRO_CLIENT"].value_counts()

NRO_CLIENT
1423425    48
817312     48
1278185    48
817591     48
1316500    48
           ..
926201     48
937830     48
932107     48
910114     48
519464     48
Name: count, Length: 331, dtype: int64

In [57]:
geodf_geral_316["NRO_CLIENT"].value_counts()

NRO_CLIENT
51139      48
1413514    48
1442243    48
1473512    48
1507013    48
           ..
814236     48
816370     48
816376     48
816382     48
1367651    48
Name: count, Length: 198, dtype: int64

In [58]:
geodf_geral_216['datetime'] = pd.to_datetime(geodf_geral_216['datetime'].copy())
geodf_geral_316['datetime'] = pd.to_datetime(geodf_geral_316['datetime'].copy())

In [59]:
geodf_geral_216.drop(columns=['CNS_ACT_FP_y'], inplace=True)
geodf_geral_316.drop(columns=['CNS_ACT_FP_y'], inplace=True)

geodf_geral_216.rename(columns={'CNS_ACT_FP_x': 'CNS_ACT_FP'}, inplace=True)
geodf_geral_316.rename(columns={'CNS_ACT_FP_x': 'CNS_ACT_FP'}, inplace=True)

In [60]:
# Plotando o consumo (coluna "CNS_ACT_FP") de um cliente específico ao longo do tempo

fig = px.line(
    geodf_geral_316[geodf_geral_316['NRO_CLIENT'] == 203475],
    x="datetime",
    y="CNS_ACT_FP",
    title="Consumo de um cliente específico ao longo do tempo"
)

fig.show()


In [61]:
# Será calculado o consumo médio dos clientes e o desvio padrão para cada mês e ano
# Cada vez que o consumo de um cliente for maior que a média + desvio padrão, 
# Ou menor que a média - desvio padrão, ele será considerado um outlier

# Guardarei os outliers em uma nova coluna chamada "desvio"
# Se o valor for "menor", o consumo do cliente é menor que a média - desvio padrão
# Se o valor for "maior", o consumo do cliente é maior que a média + desvio padrão
# Se o valor for "dentro", o consumo do cliente não é um outlier

df_std_mean_216 = pd.DataFrame()
df_std_mean_316 = pd.DataFrame()

for i in range(2019, 2023):
    for j in range(1, 13):
        df_month_216 = geodf_geral_216[(geodf_geral_216['datetime'] == pd.to_datetime(f'{i}-{j}'))]
        df_month_316 = geodf_geral_316[(geodf_geral_316['datetime'] == pd.to_datetime(f'{i}-{j}'))]

        mean_216 = df_month_216['CNS_ACT_FP'].mean()
        std_216 = np.std(df_month_216['CNS_ACT_FP'])

        mean_316 = df_month_316['CNS_ACT_FP'].mean()
        std_316 = np.std(df_month_316['CNS_ACT_FP'])

        df_std_mean_216 = pd.concat([df_std_mean_216, pd.DataFrame({'datetime': pd.to_datetime(f'{i}-{j}'), 'mean': mean_216, 'std': std_216}, index=[0])], ignore_index=True)
        df_std_mean_316 = pd.concat([df_std_mean_316, pd.DataFrame({'datetime': pd.to_datetime(f'{i}-{j}'), 'mean': mean_316, 'std': std_316}, index=[0])], ignore_index=True)

        geodf_geral_216.loc[(geodf_geral_216['datetime'] == pd.to_datetime(f'{i}-{j}')) & (geodf_geral_216['CNS_ACT_FP'] > mean_216 + std_216), 'desvio'] = 'maior'
        geodf_geral_216.loc[(geodf_geral_216['datetime'] == pd.to_datetime(f'{i}-{j}')) & (geodf_geral_216['CNS_ACT_FP'] < mean_216 - std_216), 'desvio'] = 'menor'
        geodf_geral_216.loc[(geodf_geral_216['datetime'] == pd.to_datetime(f'{i}-{j}')) & (geodf_geral_216['CNS_ACT_FP'] <= mean_216 + std_216) & (geodf_geral_216['CNS_ACT_FP'] >= mean_216 - std_216), 'desvio'] = 'dentro'

        geodf_geral_316.loc[(geodf_geral_316['datetime'] == pd.to_datetime(f'{i}-{j}')) & (geodf_geral_316['CNS_ACT_FP'] > mean_316 + std_316), 'desvio'] = 'maior'
        geodf_geral_316.loc[(geodf_geral_316['datetime'] == pd.to_datetime(f'{i}-{j}')) & (geodf_geral_316['CNS_ACT_FP'] < mean_316 - std_316), 'desvio'] = 'menor'
        geodf_geral_316.loc[(geodf_geral_316['datetime'] == pd.to_datetime(f'{i}-{j}')) & (geodf_geral_316['CNS_ACT_FP'] <= mean_316 + std_316) & (geodf_geral_316['CNS_ACT_FP'] >= mean_316 - std_316), 'desvio'] = 'dentro'

# Fazendo um dicionário contendo o número do cliente (coluna NRO_CLIENT) e quantas vezes 
# ele aparece no dataframe com a coluna "desvio" diferente de "dentro"





In [62]:

df_std_mean_216



datetime        mean         std
0  2019-01-01  153.294260   95.818510
1  2019-02-01  161.632326   99.467847
2  2019-03-01  161.171903   99.098399
3  2019-04-01  155.093051   99.175026
4  2019-05-01  151.338066  104.845415
5  2019-06-01  142.176435   73.701142
6  2019-07-01  143.557402   77.890685
7  2019-08-01  147.476435   87.309159
8  2019-09-01  140.137160   74.818960
9  2019-10-01  139.407251   80.341336
10 2019-11-01  146.547432   79.741655
11 2019-12-01  147.169184   75.414902
12 2020-01-01  146.798792   75.072440
13 2020-02-01  160.562840   89.112196
14 2020-03-01  135.719033   68.070491
15 2020-04-01  139.250755   65.594612
16 2020-05-01  464.779758  292.225339
17 2020-06-01  167.306042  153.264079
18 2020-07-01  146.330816   86.480149
19 2020-08-01  152.637160  194.300499
20 2020-09-01  150.349547   79.600609
21 2020-10-01  143.544411   82.311432
22 2020-11-01  147.092749   72.414100
23 2020-12-01  147.718429   94.258273
24 2021-01-01  154.027795   96.657849
25 2021-02-01  144.809668   74.191917
26 2021-03-01  141.081269   71.705160
27 2021-04-01  139.906344   82.297563
28 2021-05-01  141.283384   73.776712
29 2021-06-01  143.683686   74.402443
30 2021-07-01  140.035952   79.669123
31 2021-08-01  140.056495   70.659296
32 2021-09-01  131.690030   79.540711
33 2021-10-01  135.348036   73.809690
34 2021-11-01  133.812085   70.460216
35 2021-12-01  128.542296   66.806914
36 2022-01-01  133.488520   65.315684
37 2022-02-01  143.182477   83.298477
38 2022-03-01  144.486405   80.333180
39 2022-04-01  148.727492   86.566252
40 2022-05-01  131.904532   76.164762
41 2022-06-01  130.692447   75.513511
42 2022-07-01  124.907251   69.754246
43 2022-08-01  130.098489   75.361887
44 2022-09-01  123.719033   70.533454
45 2022-10-01  138.135347  100.726868
46 2022-11-01  129.383082   73.255771
47 2022-12-01  134.663746   78.040421

In [63]:
df_std_mean_316

datetime        mean         std
0  2019-01-01  172.825253  111.116221
1  2019-02-01  187.564646  143.249836
2  2019-03-01  179.060606  110.939583
3  2019-04-01  169.247980  105.464088
4  2019-05-01  173.980303  163.687108
5  2019-06-01  159.589899   99.291893
6  2019-07-01  160.779798  120.478395
7  2019-08-01  153.468687   92.968831
8  2019-09-01  160.777273  123.917302
9  2019-10-01  141.434848   76.034386
10 2019-11-01  161.802020   93.156872
11 2019-12-01  149.516667   86.245954
12 2020-01-01  158.069697  104.015390
13 2020-02-01  166.348485   97.218507
14 2020-03-01  142.191919   77.187058
15 2020-04-01  146.005051   74.998485
16 2020-05-01  480.015556  269.349017
17 2020-06-01  149.206566  107.910931
18 2020-07-01  148.253535  111.143848
19 2020-08-01  152.517222   95.506925
20 2020-09-01  153.942929   85.907538
21 2020-10-01  141.170758   89.369379
22 2020-11-01  145.797980   73.235274
23 2020-12-01  141.092929   76.194402
24 2021-01-01  161.739899  147.130836
25 2021-02-01  153.505051   97.251872
26 2021-03-01  159.203535  150.413803
27 2021-04-01  150.398485   99.946068
28 2021-05-01  145.213636   77.064226
29 2021-06-01  143.532323   80.884897
30 2021-07-01  136.873232   73.001790
31 2021-08-01  138.890404   82.124081
32 2021-09-01  134.325758   81.172113
33 2021-10-01  138.781818   77.108170
34 2021-11-01  135.043939   76.743716
35 2021-12-01  132.752020   71.881444
36 2022-01-01  139.829293   75.965621
37 2022-02-01  150.190404   80.336605
38 2022-03-01  144.496970   83.178624
39 2022-04-01  147.754040   96.393387
40 2022-05-01  133.130808   77.092003
41 2022-06-01  129.480808   86.423741
42 2022-07-01  123.843939   81.629582
43 2022-08-01  127.131818   74.083064
44 2022-09-01  122.138384   71.254492
45 2022-10-01  133.031818   80.511831
46 2022-11-01  128.495960   81.329369
47 2022-12-01  131.105556   78.526811

In [64]:
# Plotando o consumo médio +- desvio padrão de cada mês e ano

fig = px.line(
    df_std_mean_316,
    x="datetime",
    y="mean",
    title="Consumo médio +- desvio padrão de cada mês e ano"
)

fig.add_scatter(
    x=df_std_mean_316['datetime'],
    y=df_std_mean_316['mean'] + df_std_mean_316['std'],
    mode='lines',
    name='mean + std'
)

fig.add_scatter(
    x=df_std_mean_316['datetime'],
    y=df_std_mean_316['mean'] - df_std_mean_316['std'],
    mode='lines',
    name='mean - std'
)

fig.add_scatter(
    x=geodf_geral_316[geodf_geral_316['NRO_CLIENT'] == 228703]["datetime"],
    y=geodf_geral_316[geodf_geral_316['NRO_CLIENT'] == 228703]["CNS_ACT_FP"],
    mode='lines',
    name='cliente 816770'
)

fig.show()

In [65]:
# Plotando o consumo médio +- desvio padrão de cada mês e ano

fig = px.line(
    df_std_mean_216,
    x="datetime",
    y="mean",
    title="Consumo médio +- desvio padrão de cada mês e ano"
)

fig.add_scatter(
    x=df_std_mean_216['datetime'],
    y=df_std_mean_216['mean'] + df_std_mean_216['std'],
    mode='lines',
    name='mean + std'
)

fig.add_scatter(
    x=df_std_mean_216['datetime'],
    y=df_std_mean_216['mean'] - df_std_mean_216['std'],
    mode='lines',
    name='mean - std'
)

fig.add_scatter(
    x=geodf_geral_216[geodf_geral_216['NRO_CLIENT'] == 816780]["datetime"],
    y=geodf_geral_216[geodf_geral_216['NRO_CLIENT'] == 816780]["CNS_ACT_FP"],
    mode='lines',
    name='cliente 816770'
)

fig.show()

In [66]:
# Agora verei quais clientes (NRO_CLIENT) mais mais fugiram do desvio padrão,
# isso pode ser observado na coluna "desvio" dos dataframes geodf_geral_316 e geodf_geral_216
# Todos que sairam do desvio estão assinalados como "maior" ou "menor"

df_suspeitos_216 = geodf_geral_216[geodf_geral_216["desvio"] != "dentro"]
df_suspeitos_316 = geodf_geral_316[geodf_geral_316["desvio"] != "dentro"]

df_suspeitos_216

NRO_CLIENT TARIFA  CATEGORIA  CNS_ACT_FP  CNS_ACT_HP SET  ALIMENTAD  \
14        1423425   BT5B        0.0         0.0         0.0  SL        3.0   
57          48851   BT5B        0.0       252.4         0.0  SL        4.0   
64          48851   BT5B        0.0       912.5         0.0  SL        4.0   
75          48851   BT5B        0.0       229.0         0.0  LU        4.0   
116       1426757   BT5B        0.0       248.3         0.0  SL        3.0   
...           ...    ...        ...         ...         ...  ..        ...   
15873      519464   BT5B        0.0        31.7         0.0  LU        4.0   
15874      519464   BT5B        0.0        25.2         0.0  LU        4.0   
15875      519464   BT5B        0.0        41.5         0.0  LU        4.0   
15876      519464   BT5B        0.0        44.7         0.0  LU        4.0   
15882      519464   BT5B        0.0       229.7         0.0  LU        4.0   

           SED  SEC_TIPICO  quadrado   datetime desvio  
14     C-08901           1       216 2020-03-01  menor  
57     A-03646           1       216 2019-10-01  maior  
64     A-03646           1       216 2020-05-01  maior  
75     A-03646           1       216 2021-04-01  maior  
116    A-03643           1       216 2020-09-01  maior  
...        ...         ...       ...        ...    ...  
15873  A-03646           1       216 2021-10-01  menor  
15874  A-03646           1       216 2021-11-01  menor  
15875  A-03646           1       216 2021-12-01  menor  
15876  A-03646           1       216 2022-01-01  menor  
15882  A-03646           1       216 2022-07-01  maior  

[3958 rows x 12 columns]

In [67]:
suspeitos_1 = df_suspeitos_216["NRO_CLIENT"].value_counts()
suspeitos_1_df = pd.DataFrame(suspeitos_1)
suspeitos_1_df.to_csv("suspeitos_1.csv")

suspeitos_2 = df_suspeitos_316["NRO_CLIENT"].value_counts()
suspeitos_2_df = pd.DataFrame(suspeitos_2)
suspeitos_2_df.to_csv("suspeitos_2.csv")

In [68]:
df_suspeitos_216["desvio"].value_counts()

desvio
maior    2068
menor    1890
Name: count, dtype: int64

In [69]:
df_suspeitos_316["desvio"].value_counts()


desvio
maior    1093
menor    1060
Name: count, dtype: int64